In [0]:
%sql
CREATE OR REPLACE VIEW `nintendodatabricksp1okle_workspace`.`nintendo`.`best-price-oled` AS
WITH produtos AS (
    SELECT
        codigo,
        nome,
        origem

    FROM nintendodatabricksp1okle_workspace.nintendo.products
),
preco AS (
    SELECT
        codigo,
        preco
    FROM nintendodatabricksp1okle_workspace.nintendo.price
),

detalhes AS (
    SELECT
        codigo,
        oled
    FROM nintendodatabricksp1okle_workspace.nintendo.details
),

produtos_filtrados AS (
    SELECT
        produtos.codigo,
        produtos.nome,
        produtos.origem,
        preco.preco,
        detalhes.oled
    FROM produtos
    JOIN preco ON produtos.codigo = preco.codigo
    JOIN detalhes ON produtos.codigo = detalhes.codigo
    WHERE detalhes.oled = 'Sim' AND produtos.origem IN (SELECT DISTINCT origem FROM nintendodatabricksp1okle_workspace.nintendo.products)
),

menores_precos AS (
    SELECT
        codigo,
        preco AS menor_preco,
        origem
    FROM produtos_filtrados
    QUALIFY ROW_NUMBER() OVER (PARTITION BY origem ORDER BY preco ASC) = 1

)

SELECT * FROM menores_precos